In [1]:
using thesis, PRMaps
using Pandas
using Healpix
using Dates
using FileIO, JLD2

import Stripeline as Sl

In [2]:
function run(instruments, name)

    nside = 256
    sky_model = "c1s0d0"
    t_start = DateTime(2022, 1, 1, 12, 0, 0)
    days = 10

    error = [deg2rad(1.0/60), deg2rad(0.5/60), deg2rad(1.0/3600)]
    error_label = ["60arcsec", "30arcsec", "1arcsec"]

    cam_ang = Sl.CameraAngles()
    
    setup = PRMaps.Setup(
        sampling_freq_Hz = 50.0,
        total_time_s = 24. * 3600. * days
    )

    labels = ["altitude_offset" "ground_offset" "fork" "zVAX&ω_0" "zVAX&ω_90" "zVAX&ω_180" "zVAX&ω_270"]

    for (k, err) in enumerate(error)
        FINAL_RESULTS = Dict()

        # Run ideal simulation
        (ideal_dict, ideal_map) = run_fgbuster_with_error(instruments, cam_ang, Sl.TelescopeAngles(), setup, sky_model, nside, t_start)
        FINAL_RESULTS = Dict("ideal"=>ideal_dict)
        JLD2.save_object("fgbuster/"*sky_model*"/maps/"*name*"/"*error_label[k]*"_ideal.jld2", ideal_map)

        # Run errored simulation

        w2_TA = Sl.TelescopeAngles(wheel2ang_0_rad = err)
        w3_TA = Sl.TelescopeAngles(wheel3ang_0_rad = err)
        fork_TA = Sl.TelescopeAngles(forkang_rad = err)
        wω_TA_0 = Sl.TelescopeAngles(zVAXang_rad = err, ωVAXang_rad = deg2rad(0.0))
        wω_TA_90 = Sl.TelescopeAngles(zVAXang_rad = err, ωVAXang_rad = deg2rad(90.0))
        wω_TA_180 = Sl.TelescopeAngles(zVAXang_rad = err, ωVAXang_rad = deg2rad(180.0))
        wω_TA_270 = Sl.TelescopeAngles(zVAXang_rad = err, ωVAXang_rad = deg2rad(270.0))

        tel_angs = [w2_TA, w3_TA, fork_TA, wω_TA_0, wω_TA_90, wω_TA_180, wω_TA_270]

        for (i, tel_ang) in enumerate(tel_angs)
            (result_dict, result_map) = run_fgbuster_with_error(instruments, cam_ang, tel_ang, setup, sky_model, nside, t_start) 
            FINAL_RESULTS[labels[i]] = result_dict
            JLD2.save_object("fgbuster/"*sky_model*"/maps/"*name*"/"*error_label[k]*"_"*labels[i]*".jld2", result_map)
        end

        # Salvare il dizionario con tutti i risultati
        FileIO.save("fgbuster/"*sky_model*"/results_"*name*"_"*error_label[k]*".jld2", FINAL_RESULTS)
    end
end

run (generic function with 1 method)

# Run LSPe + WMAP

In [3]:
wmap = read_pickle("../instruments/wmap_instrument.pkl")
#planck = read_pickle("../instruments/planck_instrument.pkl")
#quijote = read_pickle("../instruments/quijote_instrument.pkl")
swipe = read_pickle("../instruments/lspe_swipe_instrument.pkl")
strip = read_pickle("../instruments/lspe_strip_instrument.pkl")

#strip = Pandas.query(strip, :(frequency==43.0))
wmap = Pandas.query(wmap, :(frequency==22.69))
#planck = Pandas.query(planck, :(frequency==28.4))

columns = ["instrument", "frequency", "depth_p", "depth_i"]

#instruments = concat([wmap, planck, quijote, swipe, strip])
instruments = concat([wmap, strip, swipe])

instruments = instruments[columns]
instruments

,instrument,frequency,depth_p,depth_i
0,WMAP,22.69,294.697795,293.465609
0,LSPE/Strip,43.00,130.000000,183.847763
1,LSPE/Strip,95.00,990.000000,1400.071427
2,LSPE/SWIPE,145.00,10.000000,14.142136
3,LSPE/SWIPE,210.00,17.000000,24.041631
4,LSPE/SWIPE,240.00,34.000000,48.083261


In [4]:
run(instruments, "lspe_wmap")